# The overfitting problem

![](<src/09_Table_The Overfitting.png>)

## Load the data

In [1]:
import pandas as pd

df = pd.read_excel('data/Microsoft_LinkedIn_Processed.xlsx', parse_dates=['Date'], index_col=0)
df

,Open,High,Low,Close,Volume,change_tomorrow,change_tomorrow_direction
Date,,,,,,,
2016-12-08,56.325228,56.582507,55.902560,56.058762,21220800,1.549143,UP
2016-12-09,56.214968,56.959234,56.169027,56.940857,27349400,0.321692,UP
...,...,...,...,...,...,...,...
2023-03-14,256.750000,261.070007,255.860001,260.790009,33620300,1.751806,UP
2023-03-15,259.980011,266.480011,259.209991,265.440002,46028000,3.895731,UP


## Machine Learning Model

### Separate the data

1. Target: which variable do you want to predict?
2. Explanatory: which variables will you use to calculate the prediction?

In [2]:
target = df.change_tomorrow
explanatory = df[['Open','High','Low','Close','Volume']]

### Compute the model

The following Python code will compute the numbers of the mathematical equation that we will use to calculate if the ticker goes UP or DOWN.

In [3]:
from sklearn.tree import DecisionTreeRegressor

model_dt = DecisionTreeRegressor(max_depth=15, random_state=42)
model_dt.fit(explanatory, target)

DecisionTreeRegressor(max_depth=15, random_state=42)

### Evaluate the model: compare predictions with the reality

In [4]:
from sklearn.metrics import mean_squared_error

y_pred = model_dt.predict(X=explanatory)
mean_squared_error(y_true=target, y_pred=y_pred)

1.5265197343182655

## Train test split

### Split the dataset

- Imagine we are in 2020, we can only train the data up until 31st December 2020, how good would have the model been going foward?

In [5]:
n_days = len(df.index)
n_days

1576

In [6]:
n_days_split = int(n_days*0.70)
n_days_split

1103

In [7]:
X_train, y_train = explanatory.iloc[:n_days_split], target.iloc[:n_days_split]
X_test, y_test = explanatory.iloc[n_days_split:], target.iloc[n_days_split:]

In [8]:
X_train

,Open,High,Low,Close,Volume
Date,,,,,
2016-12-08,56.325228,56.582507,55.902560,56.058762,21220800
2016-12-09,56.214968,56.959234,56.169027,56.940857,27349400
...,...,...,...,...,...
2021-04-27,256.945813,258.527306,255.511687,257.328918,31014200
2021-04-28,251.543247,251.995119,248.468708,250.050186,46903100


In [9]:
X_test

,Open,High,Low,Close,Volume
Date,,,,,
2021-04-29,250.934248,251.562910,244.588688,248.036499,40589000
2021-04-30,245.315566,248.596390,245.178047,247.712326,30945100
...,...,...,...,...,...
2023-03-14,256.750000,261.070007,255.860001,260.790009,33620300
2023-03-15,259.980011,266.480011,259.209991,265.440002,46028000


### Fit the model on train set

In [10]:
model_dt_split = DecisionTreeRegressor(max_depth=15, random_state=42)
model_dt_split.fit(X=X_train, y=y_train)

DecisionTreeRegressor(max_depth=15, random_state=42)

### Evaluate model

#### On test set

In [11]:
y_pred_test = model_dt_split.predict(X=X_test)
mean_squared_error(y_true=y_test, y_pred=y_pred_test)

9.937085384182808

#### On train set

In [12]:
y_pred_train = model_dt_split.predict(X=X_train)
mean_squared_error(y_true=y_train, y_pred=y_pred_train)

0.7335232619210788

## [ ] Backtesting

In [13]:
from backtesting import Backtest

/Users/sota/miniforge3/envs/backtesting/lib/python3.11/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

### Create library for your strategies

### Import the library

In [14]:
import strategies

### Run the backtest on `test` data

In [15]:
bt = Backtest(X_test, strategies.SimpleRegression,
              cash=10000, commission=.002, exclusive_orders=True)

results = bt.run(model=model_dt_split, limit_buy=1, limit_sell=-5)

df_results_test = results.to_frame(name='Values').loc[:'Return [%]']\
    .rename({'Values':'Out of Sample (Test)'}, axis=1)
df_results_test

,Out of Sample (Test)
Start,2021-04-29 00:00:00
End,2023-03-15 00:00:00
Duration,685 days 00:00:00
Exposure Time [%],92.811839
Equity Final [$],11034.131324
Equity Peak [$],13371.824828
Return [%],10.341313


### Run the backtest on `train` data

In [16]:
bt = Backtest(X_train, strategies.SimpleRegression,
              cash=10000, commission=.002, exclusive_orders=True)

results = bt.run(model=model_dt_split, limit_buy=1, limit_sell=-5)

df_results_train = results.to_frame(name='Values').loc[:'Return [%]']\
    .rename({'Values':'In Sample (Train)'}, axis=1)
df_results_train

,In Sample (Train)
Start,2016-12-08 00:00:00
End,2021-04-28 00:00:00
Duration,1602 days 00:00:00
Exposure Time [%],99.365367
Equity Final [$],62957.753308
Equity Peak [$],64404.1712
Return [%],529.577533


### Compare both backtests

In [17]:
df_results = pd.concat([df_results_test, df_results_train], axis=1)
df_results

,Out of Sample (Test),In Sample (Train)
Start,2021-04-29 00:00:00,2016-12-08 00:00:00
End,2023-03-15 00:00:00,2021-04-28 00:00:00
Duration,685 days 00:00:00,1602 days 00:00:00
Exposure Time [%],92.811839,99.365367
Equity Final [$],11034.131324,62957.753308
Equity Peak [$],13371.824828,64404.1712
Return [%],10.341313,529.577533


## How to solve the overfitting problem?

- Walk Forward Validation:

![](<src/10_Table_Validation Methods.png>)